The document includes an overview and description of the database's purpose and structure, a Crow's-foot diagram illustrating the relationships between tables, examples of the types of data stored, and sample queries demonstrating how to interact with the database.


GITHUB URL:  <https://github.com/cmsc-vcu/cmsc508-fa2023-hw7-EmilSalazar>


In [ ]:
#| eval: true
#| echo: false
import os
import sys
import pandas as pd
from tabulate import tabulate
from dotenv import load_dotenv
from sqlalchemy import create_engine, text
from sqlalchemy.exc import OperationalError, ProgrammingError
from IPython.display import display, Markdown

In [ ]:
#| eval: true
#| echo: false

# modify config_map to reflect credentials needed by this program
config_map = {
    'user':'HW7_USER',
    'password':'HW7_PASSWORD',
    'host':'HW7_HOST',
    'database':'HW7_DB_NAME'
}
# load and store credentials
load_dotenv()
config = {}
for key in config_map.keys():
    config[key] = os.getenv(config_map[key])
flag = False
for param in config.keys():
    if config[param] is None:
        flag = True
        print(f"Missing {config_map[param]} in .env file")
#if flag:
#    sys.exit(1)

In [ ]:
#| eval: true
#| echo: false

# build a sqlalchemy engine string
engine_uri = f"mysql+pymysql://{config['user']}:{config['password']}@{config['host']}/{config['database']}"

# create a database connection.  THIS IS THE ACTUAL CONNECTION!
try:
    cnx = create_engine(engine_uri)
except ArgumentError as e:
    print(f"create_engine: Argument Error: {e}")
    #sys.exit(1)
except NoSuchModuleError as e:
    print(f"create_engine: No Such Module Error: {e}")
    #sys.exit(1)
except Exception as e:
    print(f"create_engine: An error occurred: {e}")
    #sys.exit(1)

In [ ]:
#| echo: false
#| eval: true

# Do a quick test of the connection and trap the errors better!
try:
    databases = pd.read_sql("show databases",cnx)
except ImportError as e:
    print(f"Error: {e}")
except OperationalError as e:
    print(f"Database/SQL Error:\n{str(e)}\n")
except ProgrammingError as e:
    print(f"Programming Error:\n{str(e)}\n")
except Exception as e:
    print(f"An error occurred:\n{str(e)}\n")

## Overview and description

This database represents a company-wide resume capturing the collective skills and certifications of all the people in our company. It consists of three main tables - people, skills, and roles - which work together to find employees with specific skills, list the skills of any employee and help find gaps in the skill sets of employees in specific roles.

## Crows-foot diagram

The diagram visually represents the structure of our database, highlighting the relationships between people, their skills, and roles. It shows how the people, skills, and roles tables are interconnected via the peopleskills and peopleroles tables, illustrating a many-to-many relationship among them. This setup allows for tracking which skills each person has and what roles they occupy within the organization.


```{mermaid}
erDiagram

    people ||--o{ peopleskills : "have"
    skills ||--o{ peopleskills : "includes"
    people ||--o{ peopleroles : "occupy"
    roles ||--o{ peopleroles : "assign"

    skills {
    INT id "Primary Key"
    VARCHAR name
    VARCHAR description
    VARCHAR tag
    VARCHAR url
    VARCHAR time_commitment
    }

    people {
    INT id
    VARCHAR first_name
    VARCHAR last_name
    VARCHAR email
    VARCHAR linkedIn_url
    VARCHAR headshot_url
    VARCHAR discord_handle
    VARCHAR brief_bio
    DATE date_joined
    }

    peopleskills {
    INT id
    INT skills_id
    INT people_id
    DATE date_acquired
    }

    roles {
    INT id
    VARCHAR name
    INT sort_priority
    }

    peopleroles {
    INT id
    INT people_id
    INT role_ID
    DATE date_assigned
    }
```


## Examples of data in the database

### People table

The *people* table stores information about individuals. Each record in this table represents a person, with details about their professional lives. Some of these details include their name, linkedIn, discord, bio, and more.

Below is a list of data in the *people* table.


In [ ]:
#| echo: false
#| eval: true
sql = f"""
select * from people;
"""
## Add code to list roles table here
try:
    df = pd.read_sql(sql,cnx)
    df
except Exception as e:
    message = str(e)
    print(f"An error occurred:\n\n{message}\n\nIgnoring and moving on.")
    df = pd.DataFrame()
df

### Skills table

The *skills* table contains details about various skills. Each record is a distinct skill with a description and other relevant information. Some of these details also include a URL for more information about the skill and the time commitment needed for said skill.

Below is a list of data in the *skills* table.


In [ ]:
#| echo: false
#| eval: true
sql = f"""
select * from skills;
"""
## Add code to list roles table here
try:
    df = pd.read_sql(sql,cnx)
    df
except Exception as e:
    message = str(e)
    print(f"An error occurred:\n\n{message}\n\nIgnoring and moving on.")
    df = pd.DataFrame()
df

### Roles table

The *roles* table is designed to keep track of various roles or positions. Each record has a specific role, often used in a professional environment. Also included is the sort priority to determine the order in which roles are sorted or listed.

Below is a list of data in the *roles* table.


In [ ]:
#| echo: false
#| eval: true
sql = f"""
select * from roles;
"""
## Add code to list roles table here
try:
    df = pd.read_sql(sql,cnx)
    df
except Exception as e:
    message = str(e)
    print(f"An error occurred:\n\n{message}\n\nIgnoring and moving on.")
    df = pd.DataFrame()
df

## Sample queries

Let's explore the database!

# List skill names, tags, and descriptions ordered by name


In [ ]:
sql_skills_ordered_by_name = """
SELECT name, tag, description FROM skills ORDER BY name;
"""

In [ ]:
#| echo: false
#| eval: true
try:
    df = pd.read_sql(sql_skills_ordered_by_name,cnx)
    df
except Exception as e:
    message = str(e)
    print(f"An error occurred:\n\n{message}\n\nIgnoring and moving on.")
    df = pd.DataFrame()
df

### List people names and email addresses ordered by last_name


In [ ]:
sql_people_names_emails_ordered = """
SELECT CONCAT(first_name, ' ', last_name) AS full_name, email FROM people ORDER BY last_name;
"""

In [ ]:
#| echo: false
#| eval: true
try:
    df = pd.read_sql(sql_people_names_emails_ordered,cnx)
    df
except Exception as e:
    message = str(e)
    print(f"An error occurred:\n\n{message}\n\nIgnoring and moving on.")
    df = pd.DataFrame()
df

### List skill names of Person 1


In [ ]:
sql_skills_of_person_1 = """
SELECT s.name FROM skills s JOIN peopleskills ps ON s.id = ps.skills_id WHERE ps.people_id = 1;
"""

In [ ]:
#| echo: false
#| eval: true
try:
    df = pd.read_sql(sql_skills_of_person_1,cnx)
    df
except Exception as e:
    message = str(e)
    print(f"An error occurred:\n\n{message}\n\nIgnoring and moving on.")
    df = pd.DataFrame()
df

### List people names with Skill 6


In [ ]:
sql_people_with_skill_6 = """
SELECT CONCAT(p.first_name, ' ', p.last_name) AS full_name FROM people p JOIN peopleskills ps ON p.id = ps.people_id WHERE ps.skills_id = 6;
"""

In [ ]:
#| echo: false
#| eval: true
try:
    df = pd.read_sql(sql_people_with_skill_6,cnx)
    df
except Exception as e:
    message = str(e)
    print(f"An error occurred:\n\n{message}\n\nIgnoring and moving on.")
    df = pd.DataFrame()
df

### List people with a DEVELOPER role


In [ ]:
sql_people_with_developer_role = """
SELECT CONCAT(p.first_name, ' ', p.last_name) AS full_name FROM people p JOIN peopleroles pr ON p.id = pr.people_id JOIN roles r ON pr.role_id = r.id WHERE r.name = 'Developer';
"""

In [ ]:
#| echo: false
#| eval: true
try:
    df = pd.read_sql(sql_people_with_developer_role,cnx)
    df
except Exception as e:
    message = str(e)
    print(f"An error occurred:\n\n{message}\n\nIgnoring and moving on.")
    df = pd.DataFrame()
df

### List names and email addresses of people without skills


In [ ]:
sql_people_without_skills = """
SELECT CONCAT(p.first_name, ' ', p.last_name) AS full_name, p.email FROM people p WHERE NOT EXISTS (SELECT 1 FROM peopleskills ps WHERE ps.people_id = p.id);
"""

In [ ]:
#| echo: false
#| eval: true
try:
    df = pd.read_sql(sql_people_without_skills,cnx)
    df
except Exception as e:
    message = str(e)
    print(f"An error occurred:\n\n{message}\n\nIgnoring and moving on.")
    df = pd.DataFrame()
df

### List names and tags of unused skills


In [ ]:
sql_unused_skills = """
SELECT s.name, s.tag FROM skills s WHERE NOT EXISTS (SELECT 1 FROM peopleskills ps WHERE ps.skills_id = s.id);
"""

In [ ]:
#| echo: false
#| eval: true
try:
    df = pd.read_sql(sql_unused_skills,cnx)
    df
except Exception as e:
    message = str(e)
    print(f"An error occurred:\n\n{message}\n\nIgnoring and moving on.")
    df = pd.DataFrame()
df

### List people names and skill names with the BOSS role


In [ ]:
sql_people_skills_with_boss_role = """
SELECT CONCAT(p.first_name, ' ', p.last_name) AS full_name, s.name AS skill_name FROM people p JOIN peopleroles pr ON p.id = pr.people_id JOIN roles r ON pr.role_id = r.id JOIN peopleskills ps ON p.id = ps.people_id JOIN skills s ON ps.skills_id = s.id WHERE r.name = 'Boss';
"""

In [ ]:
#| echo: false
#| eval: true
try:
    df = pd.read_sql(sql_people_skills_with_boss_role,cnx)
    df
except Exception as e:
    message = str(e)
    print(f"An error occurred:\n\n{message}\n\nIgnoring and moving on.")
    df = pd.DataFrame()
df

### List ids and names of unused roles


In [ ]:
sql_unused_roles = """
SELECT r.id, r.name FROM roles r WHERE NOT EXISTS (SELECT 1 FROM peopleroles pr WHERE pr.role_id = r.id);
"""

In [ ]:
#| echo: false
#| eval: true
try:
    df = pd.read_sql(sql_unused_roles,cnx)
    df
except Exception as e:
    message = str(e)
    print(f"An error occurred:\n\n{message}\n\nIgnoring and moving on.")
    df = pd.DataFrame()
df

## Reflection

My experience with this assignment has been very positive. I think the assignment was surprisingly easier and more enjoyable than the previous one. I think the main reason is that since I already knew how to set everything up, I didn't have to worry about simply adding data to the database. My only complaint is how tedious the assignment was since there was a lot of data to be added but, I think it was very helpful nonetheless since there was a variety of ways we were adding it between sections. Overall, I actually learned a lot. I think this was the assignment I needed to finally understand databases and SQL better. I feel a lot better now about the project after this!